# BeautyfulSoup 네이버 블로그 스크래핑

## 블로그 포스트 스크래핑 함수 정의

In [26]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd

def get_post(query, page_num):
  url_query = quote(query)
  url = 'https://search.naver.com/search.naver?where=post&query=' + url_query
  post_df = pd.DataFrame(columns=("Title", "Date", "Blogger", "Blog URL", "Post URL", "Post"))
  idx = 0

  for _ in range(0, page_num):
    search_url = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(search_url, 'html.parser')
    posts = soup.find_all('li', {'class':'sh_blog_top'})
    

    for post in posts:
      title = post.find('a', {'class':'sh_blog_title _sp_each_url _sp_each_title'}).get('title')
      date = post.find('dd', {'class':'txt_inline'}).get_text()
      blogger = post.find('a', {'class':'txt84'}).get_text()
      blog_url = post.find('a', {'class':'txt84'}).get('href')
      post_url = post.find('a', {'class':'url'}).get('href')
      post_link = urllib.request.urlopen(post_url).read()
      post_html = BeautifulSoup(post_link, 'html.parser')

      for main_frame in post_html.select("iframe#mainFrame"):
        frame_url=  "http://blog.naver.com" + main_frame.get('src')
        post_text = urllib.request.urlopen(frame_url).read()
        post_html = BeautifulSoup(post_text, 'html.parser')
        post_content_text = ""

        for post_content in post_html.find_all('div', {'class': 'se-main-container'}):
          post_content_text = post_content.get_text()
        
        post_content_text = post_content_text.replace("\n","").replace("\t","")

        post_df.loc[idx] = [title, date, blogger, blog_url, post_url, post_content_text]
        idx +=1
        print('#', end="")
    try:
      next = soup.find("a", {'class':'next'}).get('href')
      url = "https://search.naver.com/search.naver" + next
    except:
      break

  return post_df

## 블로그 포스트 스크래핑 실행

In [27]:
query = input("검색 질의: ")
post_df = get_post(query, 10)
print("Done")

검색 질의: 컴퓨터
##########################################################################Done


In [28]:
post_df

,Title,Date,Blogger,Blog URL,Post URL,Post
0,FHD 실무를 위한 영상편집용 컴퓨터,4시간 전,원트컴 (구.마블컴),https://blog.naver.com/all_in_box,https://blog.naver.com/all_in_box?Redirect=Log...,"영상편집용 컴퓨터​오늘은 유튜브등의 플랫폼에서 대표적으로 널리 쓰이고 있는, 192..."
1,숯공장에서 온라인주문 확인할 저렴한 컴퓨터. 모니터 포함 30만원이하로.../ 라이...,어제,.,https://blog.naver.com/zxcvnm13k,https://blog.naver.com/zxcvnm13k?Redirect=Log&...,​공장에서 온라인 주문만 하면 된다고 하여... 문의한 분이다. 알고보니 대략 8년...
2,4K 작업에 최적화된 영상편집용 컴퓨터,3일 전,조인스컴 공식블로그,https://blog.naver.com/hanwoo10303,https://blog.naver.com/hanwoo10303?Redirect=Lo...,오늘은 성능과 가성비를 동시에 잡은 4K 영상편집용 컴퓨터를 소개해 드리겠습니다. ...
3,강남컴퓨터학원 꿈을 위한 한걸음!,어제,English and Life♥,https://blog.naver.com/aldus9834,https://blog.naver.com/aldus9834?Redirect=Log&...,여태 다루고 있었던 전공과는무관하지만 그래도 배우고 싶었던게 있었어요혼자 시작하려 ...
4,저렴하지만 뛰어난 성능을 갖춘 PC방 사양 조립식 컴퓨터,9시간 전,비컴의 컴퓨터이야기,https://blog.naver.com/becom5f,https://blog.naver.com/becom5f?Redirect=Log&lo...,"Overview 부담 없는 가격, 강력한 성능 가성비 피시방 조립컴퓨터. 피시방 ..."
...,...,...,...,...,...,...
69,핑크 감성으로 완성한 용산 조립컴퓨터,2020.06.26.,조인스컴 공식블로그,https://blog.naver.com/hanwoo10303,https://blog.naver.com/hanwoo10303?Redirect=Lo...,​안녕하세요! 오늘은 남자들의 로망! 핑크 색상의 조립컴퓨터를 소개해 드리겠습니다...
70,i9 10900K 두둥 등장! 초 고사양 조립 컴퓨터,2020.06.30.,비컴의 컴퓨터이야기,https://blog.naver.com/becom5f,https://blog.naver.com/becom5f?Redirect=Log&lo...,Overview 현존 최강의 CPU i9 10900K의 퍼포먼스를 느껴보세요! 안...
71,영화감상용 슬림 조립컴퓨터,2020.06.30.,용산 세븐컴 씨즌2 ...,https://blog.naver.com/seven8805,https://blog.naver.com/seven8805?Redirect=Log&...,안녕하세요~용산 조립컴퓨터 전문매장 세븐컴피씨 입니다.​I3-9100 + 8GB ...
72,10700+RTX2070S탑재 게이밍 컴퓨터,2020.06.18.,양컴 양실장의 조립...,https://blog.naver.com/combox0,https://blog.naver.com/combox0?Redirect=Log&lo...,"게이밍 조립컴퓨터​안녕하세요, 양컴입니다.오늘은 160만원대의 게이밍 시스템 소개를..."
